In [2]:
# step1 load ref length
# step2 make windows

ref_bin_list = []

window_size = 1000000

with open("./hg38_only_chromosome.fa.fai") as ref_fai:
    for ref_info in ref_fai:
        ref_info_list = ref_info.strip().split('\t')
        ref_length = int(ref_info_list[1])

        for start_idx in range(0, ref_length, window_size):
            window_start = start_idx
            window_end = start_idx + window_size

            # fix
            if window_end > ref_length:
                window_end = ref_length - 1

            ref_bin_list.append([ref_info_list[0], window_start, window_end])

In [3]:
# step3 load ref sequence
from Bio import SeqIO

ref_fa_file = SeqIO.parse(handle='./hg38_only_chromsome.fa', format='fasta')

ref_dict = {}

for ref in ref_fa_file:
    print(ref.id)
    ref_dict[ref.id] = str(ref.seq).upper()

chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX
chrY
chrM


In [4]:
# step4 calculate GC ratio
out_file = open('./hg_38_GC_ratio_1M.bed', 'w')

for window in ref_bin_list:

    chr_name, window_start, window_end = window
    window_seq = ref_dict[chr_name][window_start:window_end]

    window_G_count = window_seq.count('G')
    window_C_count = window_seq.count('C')
    window_GC_ratio = round((window_G_count + window_C_count) / 1.0 /
                            (window_end - window_start), 5)

    out_list = [
        chr_name,
        window_start,
        window_end,
        window_G_count,
        window_C_count,
        window_GC_ratio
    ]

    out_str = '\t'.join(map(str, out_list))
    out_file.write(out_str + '\n')
out_file.close()

In [6]:
# step5 calculate N count
out_file = open('./hg_38_GC_ratio_1M.bed', 'w')
total_N_count = 0

for chr_name in ref_dict:
    N_count = ref_dict[chr_name].count('N')
    total_N_count = total_N_count + N_count

print(total_N_count)

150630720


In [7]:
# import Bio
# load CTCF
in_bed_file = open('./CTCF_rep1.sort.bed', 'r')
out_fa_file = open('./CTCF_rep1.sort.bed.fa', 'w')

for line in in_bed_file:
    line_list = line.strip().split('\t')

    chr_name = line_list[0]
    region_start = int(line_list[1])
    region_end = int(line_list[2])

    ref_seq = ref_dict[chr_name][region_start:region_end]
    out_file_header = '>%s:%s-%s' % (chr_name, region_start, region_end)
    out_fa_file.write(out_file_header + '\n')
    out_fa_file.write(ref_seq + '\n')

in_bed_file.close()
out_fa_file.close()